# Book Recommendation System (User-Based Collaborative Filtering)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
# Load dataset
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
users = pd.read_csv('users.csv')

In [ ]:
# Rename columns for clarity
books.rename(columns={'Book-Title': 'Title', 'Book-Author': 'Author', 'Image-URL-M': 'Image'}, inplace=True)
ratings.rename(columns={'Book-Rating': 'Rating'}, inplace=True)

In [ ]:
# Filter users with more than 50 ratings
ratings = ratings[ratings['Rating'] > 0]
filtered_users = ratings['User-ID'].value_counts() > 50
ratings = ratings[ratings['User-ID'].isin(filtered_users[filtered_users].index)]

In [ ]:
# Merge to get book info
merged = ratings.merge(books, on='ISBN')

In [ ]:
# Create user-book matrix
user_book_matrix = merged.pivot_table(index='User-ID', columns='Title', values='Rating').fillna(0)

In [ ]:
# Compute user-user similarity
user_similarity = cosine_similarity(user_book_matrix)
np.fill_diagonal(user_similarity, 0)

In [ ]:
# Get top popular books for home page
top_books = merged.groupby('Title').agg({'Rating': ['count', 'mean']})
top_books.columns = ['RatingCount', 'AverageRating']
top_books = top_books.sort_values('RatingCount', ascending=False).head(50)
top_books = top_books.merge(books.drop_duplicates('Title')[['Title', 'Author', 'Image']], on='Title', how='left')

In [ ]:
# Save required files
pickle.dump(user_book_matrix, open('user_book_matrix.pkl', 'wb'))
pickle.dump(user_similarity, open('user_similarity.pkl', 'wb'))
pickle.dump(top_books, open('top_books.pkl', 'wb'))
pickle.dump(books, open('book_details.pkl', 'wb'))

print("✅ All files generated. You can now run the Flask app.")